### 1. INSTALL / IMPORT LIBRARIES

In [1]:
import os
# os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE" # TODO: This can introduce misscalculations. Opt for just CPU or GPU.
os.environ["OMP_NUM_THREADS"] = "1" # This is to avoid conflicts with Faiss (for MAC users)

import numpy as np
import faiss
import torch

from PyPDF2 import PdfReader

from openai import OpenAI
from transformers import CLIPProcessor, CLIPModel

# In your notebook (example):
from common_utils import embed_texts, embed_images, encode_image_to_base64, search_index, retrieve_context, call_gpt_4, extract_figures_from_pdf

/Users/olofswedberg/Documents/thesis/masters-thesis/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. CONFIGURE OPENAI & OTHER SETUPS

In [2]:
openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")  # Make sure your OpenAI key is set
)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_id).to(device)
clip_processor = CLIPProcessor.from_pretrained(model_id)

# Create a cache dictionary in memory
# response_cache = {}

### 4. PROCESS THE PDF (TEXT + IMAGES)

In [3]:
PDF_FILE = "../knowledge/catsanddogs.pdf"

text_data = []
image_data = []

# Extract text
reader = PdfReader(PDF_FILE)
num_pages = len(reader.pages)

for page_i in range(num_pages):
    page = reader.pages[page_i]
    page_text = page.extract_text()
    
    if page_text and page_text.strip():
        text_data.append({
            "text": page_text.strip(),
            "page_number": page_i + 1
        })

# Extract images
all_images = extract_figures_from_pdf(PDF_FILE)
print(f"Extracted {len(all_images)} images.")

for i, pil_img in enumerate(all_images):
    image_data.append({
        "image": pil_img,
        "image_number": i + 1
    })

Extracted 4 images.


### 5. CREATE EMBEDDINGS FOR TEXT AND IMAGE CHUNKS

In [4]:
all_metadata = []
all_embeddings = []

#### 5A. Embed all text chunks

In [5]:
texts_list = [td["text"] for td in text_data]
if len(texts_list) > 0:
    text_embeddings = embed_texts(texts_list, clip_processor, clip_model)
    for i, emb in enumerate(text_embeddings):
        all_metadata.append({
            "type": "text",
            "content": text_data[i]["text"],
            "page_number": text_data[i]["page_number"]
        })
        all_embeddings.append(emb)

#### 5B. Embed all images

In [6]:
pil_images_list = [id_["image"] for id_ in image_data]
if len(pil_images_list) > 0:
    image_embeddings = embed_images(pil_images_list, clip_processor, clip_model)
    for i, emb in enumerate(image_embeddings):
        
        # Convert PIL image to base64 once (so we can send it to GPT-4 with Vision)
        base64_str = encode_image_to_base64(image_data[i]["image"])
        
        all_metadata.append({
            "type": "image",
            # Store the base64 data directly as "content"
            "content": base64_str,
            "image_number": image_data[i]["image_number"]
        })
        all_embeddings.append(emb)

# Convert to NumPy array
all_embeddings = np.array(all_embeddings).astype('float32')
embedding_dimension = all_embeddings.shape[1]

### 6. BUILD & POPULATE FAISS

In [7]:
# Create and return a flat Faiss index of the specified dimension.
index = faiss.IndexFlatIP(embedding_dimension)

# Add embeddings to a Faiss index.
index.add(all_embeddings)

### 7. QUERY PIPELINE (RETRIEVAL + GENERATION)

In [8]:
def answer_query(user_query, top_k=3):
    """
    1. Embed the user query (as text).
    2. Retrieve top_k similar items from the PDF (text or image).
    3. Build a ChatCompletion messages list with text + images.
    4. Return GPT-4's answer.
    """
    # Step 1: Embed user query
    query_emb = embed_texts([user_query], clip_processor, clip_model)  # shape: (1, D)
    
    # Step 2: Retrieve from Faiss
    distances, faiss_indices = search_index(index, query_emb, top_k=top_k)
    retrieved_items = retrieve_context(faiss_indices, all_metadata)
    print("Distances:", distances)

    # Step 3: Build the messages payload
    # We'll pass the user's question as the first part of the content,
    # then each retrieved item (text or image) as separate parts.
    user_content = []
    
    # Add user query
    user_content.append({"type": "text", "text": f"User query: {user_query}"})
    
    # Add each retrieved item
    for item in retrieved_items:
        if item["type"] == "text":
            # Provide textual snippet
            user_content.append({
                "type": "text",
                "text": f"(page {item['page_number']}) {item['content'][:500]}..."
            })
        elif item["type"] == "image":
            # Provide the base64 image data as a data URI
            base64_str = item["content"]
            user_content.append({
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{base64_str}"
                }
            })

    # Step 4: Call GPT-4 with the full message payload
    answer = call_gpt_4(user_content)
    return answer

### 8. EXAMPLE USAGE

In [9]:
user_query_1 = "What colors are the wheels on the skateboards?"
response_1 = answer_query(user_query_1, top_k=4)
print("\nQ:", user_query_1)
print("A:", response_1)

Distances: [[0.6136132  0.4135872  0.29333058 0.29129386]]

Q: What colors are the wheels on the skateboards?
A: The wheels on the skateboards in the images are:

1. The first skateboard has silver wheels.
2. The second skateboard has orange wheels.


### 9. Defining the RAG as a Class

In [ ]:
import torch
from PyPDF2 import PdfReader
from openai import OpenAI
from transformers import CLIPProcessor, CLIPModel
from common_utils import embed_texts, embed_images, encode_image_to_base64, search_index, retrieve_context, call_gpt_4, extract_figures_from_pdf

class RAG:
    def __init__(self, pdf_path, openai_api_key):
        self.openai_client = OpenAI(api_key=openai_api_key)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_id = "openai/clip-vit-base-patch32"
        self.clip_model = CLIPModel.from_pretrained(self.model_id).to(self.device)
        self.clip_processor = CLIPProcessor.from_pretrained(self.model_id)
        self.pdf_path = pdf_path
        self.index = None
        self.all_metadata = []
        self._process_pdf()
        self._build_faiss()
    
    def _process_pdf(self):
        text_data, image_data = [], []
        reader = PdfReader(self.pdf_path)
        for i, page in enumerate(reader.pages):
            page_text = page.extract_text()
            if page_text and page_text.strip():
                text_data.append({"text": page_text.strip(), "page_number": i + 1})
        
        all_images = extract_figures_from_pdf(self.pdf_path)
        for i, pil_img in enumerate(all_images):
            image_data.append({"image": pil_img, "image_number": i + 1})
        
        texts_list = [td["text"] for td in text_data]
        if texts_list:
            text_embeddings = embed_texts(texts_list, self.clip_processor, self.clip_model)
            for i, emb in enumerate(text_embeddings):
                self.all_metadata.append({"type": "text", "content": text_data[i]["text"], "page_number": text_data[i]["page_number"]})
        
        pil_images_list = [id_["image"] for id_ in image_data]
        if pil_images_list:
            image_embeddings = embed_images(pil_images_list, self.clip_processor, self.clip_model)
            for i, emb in enumerate(image_embeddings):
                base64_str = encode_image_to_base64(image_data[i]["image"])
                self.all_metadata.append({"type": "image", "content": base64_str, "image_number": image_data[i]["image_number"]})
        
        self.all_embeddings = np.array(text_embeddings + image_embeddings).astype("float32")
        self.embedding_dimension = self.all_embeddings.shape[1]
    
    def _build_faiss(self):
        self.index = faiss.IndexFlatIP(self.embedding_dimension)
        self.index.add(self.all_embeddings)
    
    def get_most_relevant_docs(self, user_query, top_k=3):
        query_emb = embed_texts([user_query], self.clip_processor, self.clip_model)
        distances, faiss_indices = search_index(self.index, query_emb, top_k=top_k)
        return retrieve_context(faiss_indices, self.all_metadata)
    
    def generate_answer(self, user_query, retrieved_docs):
        user_content = [{"type": "text", "text": f"User query: {user_query}"}]
        for doc in retrieved_docs:
            if doc["type"] == "text":
                user_content.append({"type": "text", "text": f"(page {doc['page_number']}) {doc['content'][:500]}..."})
            elif doc["type"] == "image":
                user_content.append({"type": "image_url", "image_url": {"url": f"data:image/png;base64,{doc['content']}"}})
        return call_gpt_4(user_content)